# Auto_Price_Prediction - ML Model
-----------------------------
- Unversity of Solent : 
- BSc Final year Project :
- M.A.M Imamdeen

In [7]:
import pandas as pd
import numpy as np
import  matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pickle

from sklearn.linear_model import LinearRegression
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# Reading and Displaying the dataset

In [2]:
df = pd.read_csv("Toyota_Vehicle_data.csv")
df

,name,company,year,Price,kms_driven,fuel_type
0,Toyota,Allion,2018,9300000,8000,Petrol
1,Toyota,Allion,2018,9890000,9875,Petrol
2,Toyota,Allion,2018,9850000,10907,Petrol
3,Toyota,Allion,2008,4885000,13000,Petrol
4,Toyota,Allion,2017,8000000,22300,Petrol
...,...,...,...,...,...,...
1494,Toyota,Vitz,2007,4350000,112000,Petrol
1495,Toyota,Vitz,2008,4550000,112000,Petrol
1496,Toyota,Vitz,2007,4350000,112000,Petrol
1497,Toyota,Vitz,2007,4350000,113000,Petrol


In [8]:
df.shape


(1499, 6)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        1499 non-null   object
 1   company     1499 non-null   object
 2   year        1499 non-null   int64 
 3   Price       1499 non-null   object
 4   kms_driven  1499 non-null   int64 
 5   fuel_type   1499 non-null   object
dtypes: int64(2), object(4)
memory usage: 70.4+ KB


In [10]:
df["name"].unique()

array(['Toyota'], dtype=object)

In [11]:
df["name"].nunique()

1

## Anomalies found in "name" column:¶
* Names are pretty inconsistent.
* Names have company names attached to it.
* Some names are spam like 'Maruti Ertiga showroom condition with', 'I want to sell my car Tata Zest', etc.
* Consider first 2-3 words from the names as final name, in-order to make the categorical.

In [12]:
df["company"].unique()

array(['Allion', 'Allion ', 'Allion 240', 'Allion 260', 'Aqua',
       'Aqua G Grade', 'Aqua G grade', 'Aqua G Sport', 'Aqua S Grade',
       'Aqua X Urban', 'Avanza', 'Axio', 'Axio Fielder', 'Axio G Grade',
       'Axio WXB', 'Axio X Grade', 'Belta', 'Carina', 'Corolla 110',
       'Corolla 121', 'Corolla 141', 'Corona Premio', 'Premio',
       'Premio 240', 'Premio 260', 'Premio G Superior', 'Prius',
       'Prius 3rd Generation', 'Prius S Grade', 'Prius S Touring',
       'Roomy', 'Soluna', 'soluna', 'Tank', 'Vios', 'Vitz'], dtype=object)

In [13]:
df["company"].nunique()

36

# Anomalies found in "company" column:¶¶
* Some "company" names are incorrect.
* Ex:- 'URJENT', 'Used', '7', '2012', etc

In [14]:
df["year"].unique()

array([2018, 2008, 2017, 2016, 2014, 2013, 2011, 2010, 2009, 2007, 2012,
       2006, 2004, 2003, 2002, 2001, 2005, 2015, 2019, 2000, 2020],
      dtype=int64)

In [15]:
df["year"].nunique()

21

# #Anomalies found in "year" column:¶¶
* Many garbage/non-year values are present.
* Had to convert the dtype of the "year" column from object to int64.

In [16]:
df["Price"].unique()

array(['9300000', '9890000', '9850000', '4885000', '8000000', '7675000',
       '7900000', '7000000', '11,000,000', '13000000', '14500000',
       '11500000', '9700000', '11250000', '8250000', '8100000', '8800000',
       '7950000', '9500000', '7350000', '7100000', '7300000', '7290000',
       '7250000', '8400000', '7500000', '9100000', '6100000', '8600000',
       '7650000', '7750000', '6950000', '5575000', '5275000', '7225000',
       '5150000', '5850000', '4900000', '5900000', '8225000', '6400000',
       '7800000', '6200000', '7400000', '9000000', '4500000', '6500000',
       '6300000', '5800000', '5500000', '6000000', '5200000', '5350000',
       '5100000', '8200000', '6900000', '7390000', '9200000', '6350000',
       '7600000', '6990000', '6750000', '8575000', '6600000', '8500000',
       '5650000', '8950000', '5600000', '4975000', '7475000', '6195000',
       '4875000', '6250000', '5950000', '7200000', '7490000', '5125000',
       '5700000', '4700000', '4950000', '11450000', '83

In [17]:
df["Price"].nunique()

156

In [18]:
count = 0
for each_price in df["Price"]:
    if each_price == "Ask For Price":
        count += 1
print("No. of records in df with Price as \"Ask For Price\":", count)

No. of records in df with Price as "Ask For Price": 0


# Anomalies found in "Price" column:
* Have to look after the "Ask for Price" type values in the column.
* Had to convert the dtype of the "Price" column from object to int64 by removing the commas in-between them.

In [19]:
df["kms_driven"].unique()

array([  8000,   9875,  10907,  13000,  22300,  26000,  27000,  31000,
        34000,  41000,  47000,  49000,  54000,  56000,  60000,  71000,
        71500,  75000,  76501,  85000,  86000,  87000,  89000,  90000,
        90080,  92000,  92700,  93000,  96000,  97000, 100000, 102000,
       104000, 105000, 106000, 107000, 109000, 110000, 112000, 112500,
       113000, 115000, 116000, 117000, 118000, 120000, 121000, 124000,
       125000, 126000, 128000, 130000, 133000, 135000, 137000, 138000,
        95000,  99000, 102500, 107960, 122600, 127000, 132000, 143000,
       123000, 124500, 140000, 147000, 148000, 150000, 157000, 159000,
       160000, 174000, 183000, 190000, 200000, 230000,   5800,  15000,
        22000,  28000,  30000,  30400,  32000,  34300,  36000,  38000,
        38750,  39500,  40000,  40500,  42035,  43000,  44000,  45000,
        46000,  47100,  47500,  48000,  53000,  53300,  55000,  55227,
        55400,  57000,  58000,  60100,  62000,  62500,  63000,  63421,
      

In [20]:
df["kms_driven"].nunique()

484

In [21]:
count = 0

for each_kms in df["kms_driven"]:
    if each_kms == "Petrol":
        count += 1
        
print("No. of records in df with kms_driven as \"nan\":", df["kms_driven"].isnull().sum())
print("No. of records in df with kms_driven as \"Petrol\":", count)

No. of records in df with kms_driven as "nan": 0
No. of records in df with kms_driven as "Petrol": 0


# #Anomalies found in "kms_driven" column:
* Had to look after the "Petrol" type values in the column.
* Had to convert the dtype of the "kms_driven" column from object with "kms" in the end to int64 by removing the them.

In [22]:
df["fuel_type"].unique()


array(['Petrol', 'Hybrid', 'Electric', 'Diesel'], dtype=object)

In [23]:
df["fuel_type"].nunique()

4

In [24]:
print("\"fuel_type\" column has", df["fuel_type"].isnull().sum(), "null values.")

"fuel_type" column has 0 null values.


# Data Sanity (Clearing) & Filtering

###Data Sanity (Clearing) & Filtering
* lumn to _int64_ type

In [25]:
df["year"] = pd.to_numeric(df["year"])

### Removing all the records, having _"Price"_ == _"Ask For Price"_ 

In [26]:
df = df[df["Price"] != "Ask For Price"]

### Converting the "Price" column to pure numeric, by removing "," literals

In [27]:
df["Price"] = pd.to_numeric(df["Price"].str.replace(",", ""))

### Removing _null_ records from "fuel_type" column

In [29]:
df = df[~df["fuel_type"].isnull()]

###Considering only the first _3 words_ from the "name" column 

In [30]:
df["name"] = df["name"].str.split().str.slice(0, 3).str.join(" ")

### Removing the _"kms"_ from "kms_driven" column and replacing "," literals with ""

### Resetting the indexes of the dataframe

In [32]:
df = df.reset_index(drop=True)

In [33]:
df

,name,company,year,Price,kms_driven,fuel_type
0,Toyota,Allion,2018,9300000,8000,Petrol
1,Toyota,Allion,2018,9890000,9875,Petrol
2,Toyota,Allion,2018,9850000,10907,Petrol
3,Toyota,Allion,2008,4885000,13000,Petrol
4,Toyota,Allion,2017,8000000,22300,Petrol
...,...,...,...,...,...,...
1494,Toyota,Vitz,2007,4350000,112000,Petrol
1495,Toyota,Vitz,2008,4550000,112000,Petrol
1496,Toyota,Vitz,2007,4350000,112000,Petrol
1497,Toyota,Vitz,2007,4350000,113000,Petrol


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        1499 non-null   object
 1   company     1499 non-null   object
 2   year        1499 non-null   int64 
 3   Price       1499 non-null   int64 
 4   kms_driven  1499 non-null   int64 
 5   fuel_type   1499 non-null   object
dtypes: int64(3), object(3)
memory usage: 70.4+ KB


In [35]:
df.describe()

,year,Price,kms_driven
count,1499.000000,1.499000e+03,1499.000000
mean,2010.998666,6.403055e+06,94367.555037
std,4.823452,2.082153e+06,49223.648873
min,2000.000000,1.050000e+06,500.000000
25%,2008.000000,5.200000e+06,59583.000000
50%,2012.000000,6.100000e+06,90000.000000
75%,2015.000000,7.500000e+06,123000.000000
max,2020.000000,2.210000e+07,386400.000000


In [36]:
df["Price"].min()

1050000

In [37]:
df["Price"].quantile(0.25)

5200000.0

In [38]:
df["Price"].median()

6100000.0

In [39]:
df["Price"].quantile(0.75)

7500000.0

In [40]:
df["Price"].max()

22100000

In [41]:
find_outlier = df[df["Price"].map(str).apply(len) > 6]

In [42]:
find_outlier["Price"].min()

1050000

In [43]:
find_outlier["Price"].quantile(0.25)

5200000.0

In [44]:
find_outlier["Price"].median()

6100000.0

In [45]:
find_outlier["Price"].quantile(0.75)

7500000.0

In [46]:
find_outlier["Price"].max()

22100000

In [47]:
find_outlier[find_outlier["Price"] > find_outlier["Price"].quantile(0.75)]

,name,company,year,Price,kms_driven,fuel_type
0,Toyota,Allion,2018,9300000,8000,Petrol
1,Toyota,Allion,2018,9890000,9875,Petrol
2,Toyota,Allion,2018,9850000,10907,Petrol
4,Toyota,Allion,2017,8000000,22300,Petrol
5,Toyota,Allion,2016,7675000,26000,Petrol
...,...,...,...,...,...,...
1259,Toyota,Tank,2018,8790000,36500,Petrol
1260,Toyota,Tank,2017,7800000,50000,Petrol
1322,Toyota,Vitz,2019,8790000,1000,Petrol
1323,Toyota,Vitz,2020,8900000,1300,Petrol


In [49]:
outlier = find_outlier[find_outlier["Price"] == 11500000]
outlier

,name,company,year,Price,kms_driven,fuel_type
11,Toyota,Allion,2016,11500000,47000,Petrol
12,Toyota,Allion,2016,11500000,49000,Petrol
14,Toyota,Allion,2016,11500000,56000,Petrol


In [53]:
df = df[df["Price"] != 400000]

In [54]:
df.to_csv("Cleaned_Data.csv", index=False)

In [55]:
X = df.drop(columns = "Price")
y = df["Price"]

In [56]:
X

,name,company,year,kms_driven,fuel_type
0,Toyota,Allion,2018,8000,Petrol
1,Toyota,Allion,2018,9875,Petrol
2,Toyota,Allion,2018,10907,Petrol
3,Toyota,Allion,2008,13000,Petrol
4,Toyota,Allion,2017,22300,Petrol
...,...,...,...,...,...
1494,Toyota,Vitz,2007,112000,Petrol
1495,Toyota,Vitz,2008,112000,Petrol
1496,Toyota,Vitz,2007,112000,Petrol
1497,Toyota,Vitz,2007,113000,Petrol


In [57]:
y

0       9300000
1       9890000
2       9850000
3       4885000
4       8000000
         ...   
1494    4350000
1495    4550000
1496    4350000
1497    4350000
1498    4550000
Name: Price, Length: 1495, dtype: int64

In [58]:
X_train, X_test,y_train, y_test = train_test_split(X, y, test_size = 0.1)

In [59]:
ohenc = OneHotEncoder()
ohenc.fit(X[['name','company','fuel_type']])

OneHotEncoder()

In [60]:
column_trans = make_column_transformer((OneHotEncoder(categories = ohenc.categories_), ['name','company','fuel_type']), remainder='passthrough')

In [61]:
lnregr = LinearRegression()

In [62]:
final = make_pipeline(column_trans, lnregr)

In [63]:
final.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Toyota'], dtype=object),
                                                                            array(['Allion', 'Allion ', 'Allion 240', 'Allion 260', 'Aqua',
       'Aqua G Grade', 'Aqua G Sport', 'Aqua G grade', 'Aqua S Grade',
       'Aqua X Urban', 'Avanza', 'Axio', 'Axio Fielder', 'Axio G Grade',
       'Axio WXB', 'Axio X Gra...
       'Corolla 121', 'Corolla 141', 'Corona Premio', 'Premio',
       'Premio 240', 'Premio 260', 'Premio G Superior', 'Prius',
       'Prius 3rd Generation', 'Prius S Grade', 'Prius S Touring',
       'Roomy', 'Soluna', 'Tank', 'Vios', 'Vitz', 'soluna'], dtype=object),
                                                                            array(['Diesel', 'Electric', 'Hybrid', 'Petrol'], dtype=obje

In [64]:
final.score(X_test, y_test)

0.8923286363982491

In [65]:
y_pred = final.predict(X_test)

In [66]:
y_pred


array([ 6114268.51676285,  6891545.25128508,  6479908.82999527,
        5596890.70264411,  4174180.3057183 ,  6353696.20641816,
        5533291.24831557,  5456291.92630303,  5801797.04697049,
        7785942.85786009,  7855564.85424113,  6171839.81523454,
        6763081.09787095,  6218911.32693565,  5067590.84537756,
        5273924.94946671,  7491119.64969397,  6025600.94127941,
        3888602.76628196,  7946283.81062794,  4373516.92299485,
        7887396.68859291,  5258608.11026287,  2273279.91097045,
        5788151.13567984,  6401714.51908231,  5503959.77308595,
        7541553.46568596,  6991739.02268505,  8234473.61597848,
        6923509.46623147,  5654433.52201867,  3583086.01819015,
        5648863.76230824,  2387058.9623363 ,  7430317.41384935,
        6151295.16167307,  7472271.40183389,  6457181.2112689 ,
        6382438.03506172,  6429428.12888336,  7106898.87832999,
        8741205.11198866,  7429105.76407766,  5663194.08778656,
        3772103.55124474,  7588590.44462

In [67]:
r2_score(y_test, y_pred)

0.8923286363982491

In [68]:
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
    
    lnregr = LinearRegression()
    
    final = make_pipeline(column_trans, lnregr)
    
    final.fit(X_train, y_train)
    
    y_pred = final.predict(X_test)
    
    print(r2_score(y_test, y_pred))

0.8702631774634022
0.8576902924709318
0.8678759215465377
0.8443176863159995
0.8349443038166133
0.7923973565109186
0.8929762909060088
0.7440153688581761
0.8888184210923882
0.8644222032020029


In [69]:
scores = []
for i in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = i)
    
    lnregr = LinearRegression()
    
    final = make_pipeline(column_trans, lnregr)
    
    final.fit(X_train, y_train)
    
    y_pred = final.predict(X_test)
    
    scores.append(r2_score(y_test, y_pred))

In [70]:
np.argmax(scores)

517

In [71]:
pickle.dump(final, open("Price_Prediction.pkl", "wb"))